In [22]:
import mlx.core as mx
import mlx.nn as nn

import mlx.optimizers as optim
from mlx.utils import tree_map
from functools import partial
import mlx.core as mx
import mlx.nn as nn

from tqdm import tqdm
import numpy as np
import argparse
import random
import csv
import json

In [23]:
class gat_layer(nn.Module):
    def __init__(self, dim_proj: int, num_att_heads: int): 
        super().__init__()

        self.dim_proj = dim_proj
        self.num_att_heads = num_att_heads

        self.source_scores = mx.array([1, num_att_heads, dim_proj]) 
        self.target_scores = mx.array([1, num_att_heads, dim_proj]) 

    def __call__(self, nodes_proj, adjacency_matrix):

        node_proj = node_proj.reshape([-1, self.num_att_heads, self.dim_proj])
    
        source_scores = (node_proj * self.source_scores).sum(dim=-1)
        target_scores = (node_proj * self.target_scores).sum(dim=-1)

        neighborhood_indices = np.nonzero(adjacency_matrix) 
        masked_target_scores = mx.take(target_scores, neighborhood_indices)


In [7]:
class gat(nn.Module):
    def __init__(self, num_nodes: int, dim_embed: int, dim_proj: int, num_att_heads: int, num_layers: int, skip_connections: bool, num_out_layers: int): 
        super().__init__()
        
        total_att_size = dim_proj * num_att_heads;

        gat_layer = gat_layer(total_att_size)

        self.embed_proj = mx.linear(dim_embed, total_att_size)
        self.gat_layers = mx.Sequential([gat_layer] * num_layers)
        self.out_layers = mx.Sequential([mx.Linear(total_att_size, total_att_size)] * num_out_layers + [mx.Linear(total_att_size, 7)])

    def __call__(self, node_embeddings, adjacency_matrix):
        assert node_embeddings.shape[1] == self.dim_embed, f'Incorrect node embedding size'

        node_proj = self.embed_proj(node_embeddings);

        for layer in self.gat_layers:
            new_node_proj = layer(node_proj)
            if (self.skip_connections):
                new_node_proj += node_proj;
            node_proj = new_node_proj

        for layer in self.out_layers:
            node_embeddings = nn.leaky_relu(mx.layer(node_embeddings), .02)

        return mx.softmax(node_embeddings)

5